In [3]:
#preprocessing


import os
import shutil

# Define paths
root_dir = "../data/train_val"
output_dir = "../data/yolov8"

# Define YOLO format directories
image_dir = os.path.join(output_dir, "images")
label_dir = os.path.join(output_dir, "labels")

# Ensure directories exist
for split in ["train", "val"]:
    os.makedirs(os.path.join(image_dir, split), exist_ok=True)
    os.makedirs(os.path.join(label_dir, split), exist_ok=True)

# Class mapping
class_map = {"violence": 0, "non_violence": 1}

# Process images
for category in ["violence", "non_violence"]:
    for split in ["train", "val"]:
        input_path = os.path.join(root_dir, category, split)
        output_image_path = os.path.join(image_dir, split)
        output_label_path = os.path.join(label_dir, split)

        for img_file in os.listdir(input_path):
            if img_file.endswith(".jpg") or img_file.endswith(".png"):
                # Copy image to new structure
                shutil.copy(os.path.join(input_path, img_file), output_image_path)

                # Generate corresponding label file
                label_file = os.path.join(output_label_path, img_file.replace(".jpg", ".txt").replace(".png", ".txt"))
                with open(label_file, "w") as f:
                    f.write(f"{class_map[category]} 0.5 0.5 1.0 1.0\n")  # Placeholder full-box annotation

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # Load pre-trained model
model.train(data="../config/dataset.yaml", epochs=10, imgsz=640, batch=16)

Ultralytics 8.3.91 🚀 Python-3.12.0 torch-2.5.1 CPU (Apple M2)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=../config/dataset.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None,

train: Scanning /Users/wmwong/Documents/GitHub/ThreatSensor/data/yolov8/labels/train.cache... 384 image
val: Scanning /Users/wmwong/Documents/GitHub/ThreatSensor/data/yolov8/labels/val.cache... 125 images, 0

Plotting labels to runs/detect/train3/labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train3
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10         0G      1.127      3.682      1.982         16        640:  46%|████▌     | 11/24 [

In [ ]:
metrics = model.val()
print(metrics)

In [ ]:
results = model("test_frame.jpg")
results.show()